In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['violent_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['violent_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, 
                          learning_rate=learning_rate, 
                          depth=depth, 
                          estimators=estimators, 
                          seed=816)

In [4]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.6279122962214079, 0.6378380513079742)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['violent_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['violent_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [8]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.0015, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge19',
 'age_at_current_charge20',
 'age_at_current_charge24',
 'age_at_current_charge27',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_charges1',
 'p_violence1',
 'p_misdemeanor1',
 'p_misdemeanor3',
 'p_felassult1',
 'p_drug1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'current_violence1']

In [9]:
c_grid={'C': [0.001, 0.0015]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [10]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.0015}, {'C': 0.0015}, {'C': 0.0015}, {'C': 0.0015}, {'C': 0.0015}],
 0.6557356565145962,
 0.0011930476664589796)

### RiskSLIM

In [13]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0005, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

14

In [14]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [15]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'violent_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime= 2000, 
                                max_offset = 100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 06:59 PM | 193 rows in lookup table
02/15/20 @ 06:59 PM | ------------------------------------------------------------
02/15/20 @ 06:59 PM | runnning initialization procedure
02/15/20 @ 06:59 PM | ------------------------------------------------------------
02/15/20 @ 06:59 PM | CPA produced 2 cuts
02/15/20 @ 06:59 PM | running naive rounding on 6 solutions
02/15/20 @ 06:59 PM | best objective value: 0.4866
02/15/20 @ 06:59 PM | rounding produced 5 integer solutions
02/15/20 @ 06:59 PM | best objective value is 0.4361
02/15/20 @ 06:59 PM | running sequential rounding on 6 solutions
02/15/20 @ 06:59 PM | best objective value: 0.4866
02/15/20 @ 06:59 PM | sequential rounding produced 3 integer solutions
02/15/20 @ 06:59 PM | best objective value: 0.2954
02/15/20 @ 06:59 PM | polishing 8 solutions
02/15/20 @ 06:59 PM | best objective value: 0.2954
02/15/20 @ 06:59 PM | polishing produced 5 integer solutions
02/15/20 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 06:59 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1416.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 191592  5422        cutoff              0.1353        0.1349   771676    0.33%           rho_5 D 191592  72848     21
 196377  3719        0.1353     6        0.1353        0.1350   784019    0.24%          rho_12 D 196377 120362     25
 200859   969        cutoff              0.1353        0.1352   794904    0.08%        alpha_14 U 200859 200858     27
Elapsed time = 36.80 sec. (41254.41 ticks, tree = 1.53 MB, solutions = 10)

User cuts applied:  391

Root node processing (before b&c):
  Real time             =    0.08 sec. (1.52 ticks)
Sequential b&c:
  Real time             =   36.92 sec. (41545.01 ticks)
                          ------------
Total (root+branch&cut) =   37.00 sec. (41546.53 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 07:00 PM | 193 rows in lookup table
02/15/20 @ 07:00 PM | ------------------------------------------------------------
02/15/20 @ 07:00 PM | runnning initialization procedure
02/15/20 @ 07:00 PM | ---------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:00 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1429.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Elapsed time = 34.66 sec. (41255.23 ticks, tree = 2.96 MB, solutions = 6)
 211024  5974        cutoff              0.1366        0.1361   740915    0.33%          rho_14 U 211024 146556     26
 215728  3991        cutoff              0.1366        0.1363   752001    0.23%           rho_3 D 215728 131568     27
 220438  1684        0.1365     7        0.1366        0.1364   762362    0.11%          rho_14 U 220438 131743     23

User cuts applied:  398

Root node processing (before b&c):
  Real time             =    0.08 sec. (1.79 ticks)
Sequential b&c:
  Real time             =   37.23 sec. (44628.04 ticks)
                          ------------
Total (root+branch&cut) =   37.31 sec. (44629.83 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 07:00 PM | 193 rows in lookup table
02/15/20 @ 07:00 PM | ------------------------------------------------------------
02/15/20 @ 07:00 PM | runnning initialization procedure
02/15/20 @ 07:00 PM | ----------------------

...

 205055 24596        0.1370     8        0.1373        0.1352   851726    1.49%           rho_4 U 205055 205053     47
 209254 23797        cutoff              0.1373        0.1353   864458    1.45%        alpha_14 U 209254 209252     40
Elapsed time = 42.98 sec. (41256.51 ticks, tree = 6.35 MB, solutions = 8)
 213482 23688        0.1356    11        0.1373        0.1353   878536    1.42%           rho_8 U 213482 213481     43
 217665 23375        0.1354    11        0.1373        0.1354   892629    1.38%           rho_0 U 217665 217660     29
 221926 23468        cutoff              0.1373        0.1354   906434    1.36%        alpha_12 U 221926 221923     44
 226113 22811        0.1355     1        0.1373        0.1355   920741    1.32%           rho_0 U 226113 226111     28
 230296 22591        cutoff              0.1373        0.1355   934264    1.28%         alpha_2 U 230296  90047     55
 234631 22524        cutoff              0.1373        0.1356   947661    1.24%         alpha

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:25 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1430.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 135525 13776        0.1358     8        0.1369        0.1354   479369    1.08%           rho_8 D 135525 106761     33
 138738 13702        0.1355    10        0.1369        0.1355   489584    1.05%           rho_8 U 138738 138714     36
 141896 13162        0.1365     1        0.1369        0.1355   499412    1.01%           rho_5 U 141896 133529     25
 145210 12829        0.1359     8        0.1369        0.1356   509045    0.97%          rho_10 U 145210 145209     32
 149050 12542        0.1356     6        0.1369        0.1356   518533    0.94%           rho_5 D 149050 149048     26
 152385 12156        cutoff              0.1369        0.1357   528174    0.89%         alpha_2 U 152385 152383     44
Elapsed time = 37.25 sec. (41260.19 ticks, tree = 3.47 MB, solutions = 15)
 156532 12723        0.1357     8        0.1369        0.1357   535945    0.88%           rho_0 U 156532 156522     41
 159717 12212        cutoff              0.1369        0.1358   545434    0.83%         alph

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:26 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1432.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 159779  9382        cutoff              0.1371        0.1360   748276    0.78%           rho_7 U 159779  67866     37
Elapsed time = 43.84 sec. (41257.48 ticks, tree = 2.74 MB, solutions = 7)
 163457  8481        cutoff              0.1371        0.1361   761156    0.70%           rho_7 U 163457 163456     22
 167088  7215        0.1369     8        0.1371        0.1362   773718    0.61%          rho_14 N 167088 167086     30
 170975  6487        0.1363     2        0.1371        0.1363   784962    0.53%           rho_5 U 170975 170973     32
 174704  4963        cutoff              0.1371        0.1365   796107    0.42%           rho_8 U 174704 158537     27
 178665  3773        cutoff              0.1371        0.1366   806808    0.33%           rho_5 D 178665 165090     21
 182465  2124        cutoff              0.1371        0.1368   817071    0.19%           rho_7 U 182465 182464     35

User cuts applied:  479

Root node processing (before b&c):
  Real time             =    0.1

In [16]:
np.mean(riskslim_summary['KY_validation']), np.mean(riskslim_summary['FL_score'])

(0.7386183087649036, 0.6064514090736762)

In [17]:
#### save results
summary_violent2_ky_inter_model = {"cart": cart_summary,
                                   "ebm": ebm_summary,
                                   "stumps": stumps_summary,
                                   "riskslim": riskslim_summary}

In [18]:
path = "./results/interpretable/two-year/"
results = [["Violent",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'two-year-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)